<a href="https://colab.research.google.com/github/Trickshotblaster/nn-practices/blob/main/TrigramLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import math

In [2]:
import requests

url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
url = 'https://raw.githubusercontent.com/karpathy/makemore/master/names.txt'
url = 'https://raw.githubusercontent.com/Trickshotblaster/nn-practices/main/TrigramLM.ipynb'
response = requests.get(url)

with open('file.txt', 'wb') as f:
  f.write(response.content)

In [3]:
with open('file.txt', 'r') as data:
  dataset = data.readlines()

with open('file.txt', 'r') as data:
  raw_data = data.read()


In [4]:
raw_data[:10]

'{\n  "nbfor'

In [5]:
dataset = raw_data.split("\n")

In [6]:
dataset[:10]

['{',
 '  "nbformat": 4,',
 '  "nbformat_minor": 0,',
 '  "metadata": {',
 '    "colab": {',
 '      "provenance": [],',
 '      "include_colab_link": true',
 '    },',
 '    "kernelspec": {',
 '      "name": "python3",']

In [7]:
vocab = set(raw_data)

In [8]:
vocab_len = len(vocab)

In [9]:
vocab_len

84

In [10]:
stoi = {char:i for i, char in enumerate(vocab)}

In [11]:
itos = {i:char for i, char in enumerate(vocab)}

In [12]:
import torch

In [13]:
import torch.nn.functional as functional

In [14]:
# make dataset
xs, ys = [], []
for data in dataset:
  for ch1, ch2, ch3 in zip(data, data[1:], data[2:]):
    xs.append([stoi[ch1], stoi[ch2]])
    ys.append(stoi[ch3])

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
num

29898

In [15]:
ws = torch.randn((vocab_len*2, vocab_len))
ws.requires_grad = True
ws.shape

torch.Size([168, 84])

In [16]:
xenc = functional.one_hot(xs[0], num_classes=vocab_len).float().flatten()
xenc @ ws

tensor([ 1.2902, -1.0448, -0.4287,  2.6370,  0.7902,  1.8763,  1.7406,  0.1233,
         0.4230, -0.0459,  0.9820,  1.2457,  1.2672, -1.6317,  0.5418,  1.6025,
        -1.1339, -1.7975, -1.1677, -1.0376, -1.5510,  1.3972,  0.9162, -1.7395,
        -0.1500, -0.6401, -3.6267,  1.2203, -1.0166,  1.2318,  1.0518,  1.0120,
        -0.0308,  0.8081,  4.1057,  0.3306, -2.2938,  0.6392, -1.9167, -1.7486,
        -0.0802,  0.0229, -0.1797, -1.2204,  0.9249,  0.0404, -1.0170,  0.4987,
        -0.5868, -1.1132, -2.7369, -0.2905, -2.0242,  1.2513,  0.5900, -3.2943,
        -0.0060, -2.5865,  1.1640, -0.3444, -0.2656, -0.0652, -1.7462,  1.2464,
         2.6086, -0.2733, -2.9515, -0.3248,  0.4784, -1.4727,  1.5851, -1.8234,
        -0.4941, -0.2109,  0.7347,  0.3523,  0.8111,  1.0197,  0.7730,  1.0821,
        -1.3962, -1.3378, -0.0523, -0.6386], grad_fn=<SqueezeBackward4>)

In [17]:
(xenc @ ws).shape

torch.Size([84])

In [18]:
num_epochs = 100
lr = 50

In [19]:
torch.randn((3, 1))

tensor([[-4.0799e-01],
        [ 7.8032e-01],
        [-2.7628e-04]])

In [21]:
for epoch in range(num_epochs):
    xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
    logits = xenc @ ws
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True)
    loss = -probs[torch.arange(int(num / 2)), ys].log().mean() + (ws.mean() * 0.01)
    print(loss.item())

    ws.grad = None
    loss.backward()
    ws.data += ws.grad * -lr

2.6528780460357666
2.2465128898620605
1.9401931762695312
1.830486536026001
1.8050857782363892
2.016796827316284
3.191267251968384
2.566632032394409
2.165956974029541
1.8682912588119507
1.7715778350830078
1.7477238178253174
2.0528206825256348
3.348189353942871
2.6920411586761475
2.279139757156372
1.9513193368911743
1.8941682577133179
2.1152594089508057
1.7816951274871826
2.1447243690490723
3.613374948501587
2.898707151412964
2.464660882949829
2.105971097946167
1.964372992515564
2.115729570388794
1.7882946729660034
1.9539504051208496
3.036581516265869
2.485316276550293
2.0953667163848877
1.80934476852417
1.7199320793151855
1.6941883563995361
1.8263765573501587
2.6621742248535156
2.1984660625457764
1.849191427230835
1.8120577335357666
2.2026803493499756
1.8201344013214111
2.067568778991699
3.55387282371521
2.8189685344696045
2.387122392654419
2.0307488441467285
1.8686105012893677
1.9752391576766968
1.6844390630722046
1.8741793632507324
2.9126222133636475
2.3812646865844727
2.0094058513641

In [22]:
xs[:10]

tensor([[30, 30],
        [30, 60],
        [60, 26],
        [26,  6],
        [ 6, 37],
        [37,  7],
        [ 7, 33],
        [33, 58],
        [58, 64],
        [64, 53]])

In [23]:
def forward(xs):
  xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
  logits = xenc @ ws
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdims=True)
  outchar = itos[torch.multinomial(probs, num_samples=1).item()]
  return outchar

def prompt(ins):
  out = ins
  for char in range(500):
    last_two = ins[-2:]
    xs = torch.tensor([[stoi[last_two[0]], stoi[last_two[1]]]])
    outs = forward(xs)
    out += outs
  return out

prompt("Rom")

'Rome(_ee//i/eee<eee)ea/eecaaeea_aeeee/,_i/_eea__:_eepe_ee//aia(/eieUeaeaef"___[e_p"e#yeaa(ee/e/e_eH/_ee"e_snk"-(_/eaeVaa(//_aeaeaaea_ee/=eieiUe/ap]ee_a/a//eeiese0ee_e/_e/(ai/aey /e(e_ee/e\\ea_eear/(i__[_ieaaeeee__(eeuee/1pa8"i/eaee(ee"_e_ea(aa_HRea_/gte/neieea_a=eepea__eeeeLeii_eepeehHe_e\\_eae/ee"[_eee_(wa_\n_e__gp_eeae_/maep_e_/a(\\eee/e__ee__xi/_ai(e_/e/_aie_eea=/e__e)e+e/ip(/eiee(eaei_i/#/eea/(ee_apeepe=_aeeea//[ep/L{e/_ii/aei/_ne_eae_/eie"N<e(aeaa(LaeiieaeeeG_aeaaeag(arsea_ae_eaeea(e{eepeiapeeeaey'